In [1]:
# import the required libraries
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr



In [25]:
# Load environment variables
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables")
else:
    print("OpenAI API key loaded successfully")




OpenAI API key loaded successfully


In [2]:
# setup openai
openai = OpenAI()
MODEL = "gpt-4.1-mini"



In [3]:
system_message = "You are a helpful assistant."

In [4]:
def chat(message, history):
    
    # add system prompt to message list
    messages = [{"role": "system", "content": system_message},]
    
    # Add previous messages to message list
    for turn in history:
        # Case 1: turn is a list/tuple (standard older Gradio)
        if isinstance(turn, (list, tuple)) and len(turn) >= 2:
            u, a = turn[0], turn[1]
            messages.append({"role": "user", "content": str(u)})
            messages.append({"role": "assistant", "content": str(a)})
            
        # Case 2: turn is a dictionary (newer Gradio sometimes uses this, even without type='messages' explicit)
        elif isinstance(turn, dict):
            # If it's a dict, it might be in format {'role': 'user', 'content': '...'} or similar
            # We'll just pass it through if compatible or try to adapt
            if 'role' in turn and 'content' in turn:
                messages.append(turn)

    # Add current message to message list
    messages.append({"role": "user", "content": message})

    # Get response from model
    stream = openai.chat.completions.create(
        model = MODEL,
        messages = messages,
        stream = True,
    )

    # Process response
    results = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            results += chunk.choices[0].delta.content
            yield results

In [5]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
